In [63]:
import pandas as pd
import osmnx as ox
import numpy as np
import json

In [ ]:
cities_pl = ['Warszawa', 'Kraków', 'Wrocław', 'Poznań', 'Szczecin']

all_nodes_pl = pd.DataFrame()
all_edges_pl = pd.DataFrame()

for city in cities_pl:
    G =ox.graph.graph_from_place(f"{city}, Poland", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_pl = pd.concat([all_nodes_pl, gdf_nodes])
    all_edges_pl = pd.concat([all_edges_pl, gdf_edges])


In [ ]:
cities_usa = ['Seattle', 'New York City']

all_nodes_usa = pd.DataFrame()
all_edges_usa = pd.DataFrame()

for city in cities_usa:
    G =ox.graph.graph_from_place(f"{city}, United States", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_usa = pd.concat([all_nodes_usa, gdf_nodes])
    all_edges_usa = pd.concat([all_edges_usa, gdf_edges])

In [ ]:
all_attributes = {}
for col in ['highway', 'access', 'junction', 'bridge', 'tunnel']:
    edges_usa = all_edges_usa[col].dropna().unique()
    edges_pl = all_edges_pl[col].dropna().unique()
    all_edges = set(np.concatenate((edges_pl, edges_usa)))
    all_attributes[f'edges_{col}'] = list(all_edges)

     
nodes_highway_usa = all_nodes_usa['highway'].dropna().unique()
nodes_highway_pl = all_nodes_pl['highway'].dropna().unique()   
all_nodes_highway = set(np.concatenate((nodes_highway_pl, nodes_highway_usa)))
all_attributes['nodes_highway'] = list(all_nodes_highway)

with open('../data/osmnx_attributes.json', 'w') as f:
    json.dump(all_attributes, f)
    

In [64]:
with open('../data/osmnx_attributes.json', 'r') as f:
    all_attributes = json.load(f)

In [67]:
len(all_attributes['nodes_highway'])

21

In [69]:
pd.read_csv('../data/downstream_tasks/airbnb/new_york/New York City_all_tags_one_hot_encoded.csv')

,osmid,geometry,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_hangar,aeroway_helipad,aeroway_heliport,aeroway_holding_position,aeroway_jet_bridge,...,waterway_fuel,waterway_link,waterway_lock_gate,waterway_put_in,waterway_river,waterway_sluice_gate,waterway_stream,waterway_tidal_channel,waterway_waterfall,waterway_weir
0,45197277,POINT (-73.8578336 40.7716277),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,45197679,POINT (-73.8593868 40.772357),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,45198995,POINT (-73.8629403 40.7740573),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,45199539,POINT (-73.8660382 40.775539),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,45238505,POINT (-73.8717844 40.7782121),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381066,1263799071,"LINESTRING (-73.7898223 40.8819983, -73.786796...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1381067,1268054263,"LINESTRING (-73.9235683 40.7200564, -73.923615...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1381068,1268438022,"LINESTRING (-74.2179491 40.5066119, -74.218021...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1381069,1271580479,"LINESTRING (-74.2200643 40.516824, -74.2200405...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
